<center><h1> MCD - Ciencia de Datos Responsable </h1><center>
<center><h2> Tutorial 6: AIF360 Toolkit<h2></center>


![AIF360](https://opengraph.githubassets.com/4b7ff9fccf4e6cafd9e59f42921b31a0f7f6696b009c8519e584be278094769d/Trusted-AI/AIF360)

In [ ]:
#@title Instalar la librería
%%capture
!pip install aif360['all']

In [ ]:
#@title Importación de librerías


# elementales

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

import seaborn as sns

# sklearn

from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, plot_roc_curve, recall_score, precision_score, f1_score, roc_auc_score, roc_curve

# aif360

from aif360.sklearn.datasets import fetch_german
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.datasets import BinaryLabelDataset, StandardDataset



Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


## Obtención y manipulación de los datos

Estaremos utilizando el dataset 'German Credit Data'.

In [ ]:
# Obtenemos los datos.

X, y = fetch_german()
X.head()

checking_status  duration  \
sex    age   foreign_worker                             
male   aged  yes                         <0       6.0   
female young yes                   0<=X<200      48.0   
male   aged  yes                no checking      12.0   
             yes                         <0      42.0   
             yes                         <0      24.0   

                                             credit_history  \
sex    age   foreign_worker                                   
male   aged  yes             critical/other existing credit   
female young yes                              existing paid   
male   aged  yes             critical/other existing credit   
             yes                              existing paid   
             yes                         delayed previously   

                                         purpose  credit_amount  \
sex    age   foreign_worker                                       
male   aged  yes                        radio/tv         1169.0   
female young yes                        radio/tv         5951.0   
male   aged  yes                       education         2096.0   
             yes             furniture/equipment         7882.0   
             yes                         new car         4870.0   

                               savings_status employment  \
sex    age   foreign_worker                                
male   aged  yes             no known savings        >=7   
female young yes                         <100     1<=X<4   
male   aged  yes                         <100     4<=X<7   
             yes                         <100     4<=X<7   
             yes                         <100     1<=X<4   

                             installment_commitment other_parties  \
sex    age   foreign_worker                                         
male   aged  yes                                4.0          none   
female young yes                                2.0          none   
male   aged  yes                                2.0          none   
             yes                                2.0     guarantor   
             yes                                3.0          none   

                             residence_since  ...   age  other_payment_plans  \
sex    age   foreign_worker                   ...                              
male   aged  yes                         4.0  ...  67.0                 none   
female young yes                         2.0  ...  22.0                 none   
male   aged  yes                         3.0  ...  49.0                 none   
             yes                         4.0  ...  45.0                 none   
             yes                         4.0  ...  53.0                 none   

                              housing existing_credits                 job  \
sex    age   foreign_worker                                                  
male   aged  yes                  own              2.0             skilled   
female young yes                  own              1.0             skilled   
male   aged  yes                  own              1.0  unskilled resident   
             yes             for free              1.0             skilled   
             yes             for free              2.0             skilled   

                            num_dependents  own_telephone foreign_worker  \
sex    age   foreign_worker                                                
male   aged  yes                       1.0            yes            yes   
female young yes                       1.0           none            yes   
male   aged  yes                       2.0           none            yes   
             yes                       2.0           none            yes   
             yes                       2.0           none            yes   

                                sex marital_status  
sex    age   foreign_worker                         
male   aged  yes               male         single  
female youn

In [ ]:
# Los manipularemos para dejarlos en el formato habitual.

df = pd.concat([X,y], axis=1).drop(columns=['sex',  'foreign_worker']).rename(columns={'age': 'age_numeric'}).reset_index()
df.head()

,sex,age,foreign_worker,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,...,property_magnitude,age_numeric,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,marital_status,credit-risk
0,male,aged,yes,<0,6.0,critical/other existing credit,radio/tv,1169.0,no known savings,>=7,...,real estate,67.0,none,own,2.0,skilled,1.0,yes,single,good
1,female,young,yes,0<=X<200,48.0,existing paid,radio/tv,5951.0,<100,1<=X<4,...,real estate,22.0,none,own,1.0,skilled,1.0,none,div/dep/mar,bad
2,male,aged,yes,no checking,12.0,critical/other existing credit,education,2096.0,<100,4<=X<7,...,real estate,49.0,none,own,1.0,unskilled resident,2.0,none,single,good
3,male,aged,yes,<0,42.0,existing paid,furniture/equipment,7882.0,<100,4<=X<7,...,life insurance,45.0,none,for free,1.0,skilled,2.0,none,single,good
4,male,aged,yes,<0,24.0,delayed previously,new car,4870.0,<100,1<=X<4,...,no known property,53.0,none,for free,2.0,skilled,2.0,none,single,bad


In [ ]:
df = df.rename(columns={'age': 'age_cat', 'age_numeric': 'age'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   sex                     1000 non-null   category
 1   age_cat                 1000 non-null   category
 2   foreign_worker          1000 non-null   category
 3   checking_status         1000 non-null   category
 4   duration                1000 non-null   float64 
 5   credit_history          1000 non-null   category
 6   purpose                 1000 non-null   category
 7   credit_amount           1000 non-null   float64 
 8   savings_status          1000 non-null   category
 9   employment              1000 non-null   category
 10  installment_commitment  1000 non-null   float64 
 11  other_parties           1000 non-null   category
 12  residence_since         1000 non-null   float64 
 13  property_magnitude      1000 non-null   category
 14  age                     1

### Descripción de las columnas.

**Atributos Numéricos**

- `duration`: Duración en meses. Rango (4,72).
- `credit_amount`: Cantidad de crédito solicitada. Rango (250, 18424) en DM - Marco alemán.
- `installment_commitment`: Tasa de cuota en porcentaje del ingreso disponible. Rango (1,4).
- `residence_since`: Tiempo de residencia actual. Rango (1,4).
- `age`: Edad en años. Rango (19, 75).
- `existing_credits`: Número de créditos existentes en este banco. Rango (1,4) en DM - Marco alemán.
- `num_dependents`: Número de personas responsables de proveer el mantenimiento. Rango (1,2).

**Atributos Categóricos**

- `checking_status`: Valores `'0<=X<200', '<0', '>=200', 'no checking'`.
- `credit_history`: Historial crediticio del solicitante. Valores `['all paid', 'critical/other existing credit', 'delayed previously', 'existing paid', 'no credits/all paid']`.
- `purpose`: Motivo por el cual el solicitante solicitó un préstamo. Valores `['business', 'domestic appliance', 'education', 'furniture/equipment', 'new car', 'other', 'radio/tv', 'repairs', 'retraining', 'used car']`.
- `savings_status`: Cuenta de ahorros/bonos. Valores `['100<=X<500', '500<=X<1000', '<100', '>=1000', 'no known savings']`.
- `employment`: Empleo actual desde (en años). Valores `['1<=X<4', '4<=X<7', '<1', '>=7', 'unemployed']`.
- `other_parties`: Otros deudores / garantes. Valores `['co applicant', 'guarantor', 'none']`.
- `property_magnitude`: Bienes del solicitante. Valores `['car', 'life insurance', 'no known property', 'real estate']`.
- `other_payment_plans`: Otros planes de pago a plazos. Valores `['bank', 'none', 'stores']`.
- `housing`: Situación de vivienda del solicitante. Valores `['for free', 'own', 'rent']`.
- `job`: Categorías de empleo definidas por el banco. Valores `['high qualif/self emp/mgmt', 'skilled', 'unemp/unskilled non res', 'unskilled resident']`.
- `own_telephone`: Si hay un teléfono registrado a nombre del cliente. Valores `['none', 'yes']`.
- `foreign_worker`: **Atributo protegido**. Valores `['no', 'yes']`.
- `sex`: **Atributo protegido**. Valores `['female', 'male']`.
- `marital_status`: Estado civil. Valores `['div/dep/mar', 'div/sep', 'mar/wid', 'single']`.

**Etiqueta**

- `credit-risk`: `'good'` (favorable) o `'bad'` (desfavorable).

## Pre-procesamiento

Haremos el pre-procesamiento de nuestros datos para poder trabajar con los algoritmos.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   sex                     1000 non-null   category
 1   age_cat                 1000 non-null   category
 2   foreign_worker          1000 non-null   category
 3   checking_status         1000 non-null   category
 4   duration                1000 non-null   float64 
 5   credit_history          1000 non-null   category
 6   purpose                 1000 non-null   category
 7   credit_amount           1000 non-null   float64 
 8   savings_status          1000 non-null   category
 9   employment              1000 non-null   category
 10  installment_commitment  1000 non-null   float64 
 11  other_parties           1000 non-null   category
 12  residence_since         1000 non-null   float64 
 13  property_magnitude      1000 non-null   category
 14  age                     1

In [ ]:
# Binarización de algunas variables.

df['sex'] = df['sex'].map({'male': 1, 'female': 0})
df['age_cat'] = df['age_cat'].map({'aged': 1, 'young': 0})
df['foreign_worker'] = df['foreign_worker'].map({'no': 1, 'yes': 0})
df['credit-risk'] = df['credit-risk'].map({'good': 1, 'bad': 0})

In [ ]:
# Separamos X e Y.

X = df.loc[:, df.columns != 'credit-risk']
y = df.loc[:, df.columns == 'credit-risk']

In [ ]:
df1 = X.copy()

In [ ]:
# Obtenemos variables dummies

ignore = ['sex', 'age_cat', 'foreign_worker']
for catcol in df1.select_dtypes(include='category').columns:
  if catcol in ignore:
    pass
  else:
    dummies = pd.get_dummies(X[catcol])
    X = pd.concat([X, dummies], axis=1).drop(columns=[catcol])

X.head()

,sex,age_cat,foreign_worker,duration,credit_amount,installment_commitment,residence_since,age,existing_credits,num_dependents,...,unemp/unskilled non res,unskilled resident,skilled,high qualif/self emp/mgmt,none,yes,div/dep/mar,div/sep,mar/wid,single
0,1,1,0,6.0,1169.0,4.0,4.0,67.0,2.0,1.0,...,0,0,1,0,0,1,0,0,0,1
1,0,0,0,48.0,5951.0,2.0,2.0,22.0,1.0,1.0,...,0,0,1,0,1,0,1,0,0,0
2,1,1,0,12.0,2096.0,2.0,3.0,49.0,1.0,2.0,...,0,1,0,0,1,0,0,0,0,1
3,1,1,0,42.0,7882.0,2.0,4.0,45.0,1.0,2.0,...,0,0,1,0,1,0,0,0,0,1
4,1,1,0,24.0,4870.0,3.0,4.0,53.0,2.0,2.0,...,0,0,1,0,1,0,0,0,0,1


In [ ]:
# Datos de entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=908)

In [ ]:
# No utilizaremos la edad

X_train.drop(columns=['age'], inplace=True)
X_test.drop(columns=['age'], inplace=True)

## ¿Discriminación en nuestros datos?

Una forma de medir la discriminación en nuestros datos es a través de AIF360, que nos facilita el cálculo de métricas. Para ello debemos seguir una serie de pasos para instanciar un BinaryLabelDatasetMetric.

- #### **Statistical Parity Difference**.

Corresponde a la diferencia entre la probabilidad de pertenecer a la clase positiva dado que se pertenece a la clase no privilegiada y  la probabilidad de pertenecer a la clase positiva dado que se pertenece a la clase privilegiada, es decir:

$SP = Pr(Y = 1 | D = \text{unprivileged})- Pr(Y = 1 | D = \text{privileged})$

Valores negativos de esta métrica podrían indicar la presencia de sesgo.

- #### **Disparate Impact**.

Es una métrica para medir fairness que compara la proporción de individuos que reciben un resultado positivo para el grupo privilegiado y el grupo no privilegiado. El cálculo es la proporción del grupo no privilegiado que recibe un resultado positivo dividida por la proporción del grupo privilegiado que recibe un resultado positivo.


$DI = \frac{Pr(Y = 1 | D = \text{unprivileged})}{Pr(Y = 1 | D = \text{privileged})}$



In [ ]:
# Necesitamos los datos de entrenamiento en conjunto con la etiqueta.

df_train = X_train.copy()
df_train['credit-score'] = y_train

### Discriminación por edad

In [ ]:
# Debemos definir nuestros grupos privilegiados y no privilegiados.

privileged_groups = [{'age_cat': 1}]
unprivileged_groups = [{'age_cat': 0}]

In [ ]:
# Instanciamos BinaryLabelDataset con edad como atributo protegido

attributes_params = dict(protected_attribute_names=['age_cat'], label_names=['credit-score'])
dt_train = BinaryLabelDataset(df=df_train, **attributes_params)

In [ ]:
# Finalmente, la instancia de BinaryLabelDatasetMetric con la edad como atributo protegido.

metric_age_train = BinaryLabelDatasetMetric(dt_train,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

#### Métricas

In [ ]:
print(f'Train Statistical Parity Difference (age): {metric_age_train.mean_difference()}')

Train Statistical Parity Difference (age): -0.13278706925580863


In [ ]:
from aif360.explainers import MetricTextExplainer

In [ ]:
MetricTextExplainer(metric_age_train).mean_difference()

'Mean difference (mean label value on unprivileged instances - mean label value on privileged instances): -0.13278706925580863'

In [ ]:
MetricTextExplainer(metric_age_train).disparate_impact()

'Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.8178037886955184'

**Pregunta**: Las métricas nos indican que existe sesgo con respecto a la edad en nuestros datos de entrenamiento. Supongamos que entrenamos un modelo con estos datos. ¿Que sucederá con estos sesgos dentro del modelo? ¿Serán replicados, ignorados o amplificados?

In [ ]:
# Regresión logística

lr = LogisticRegressionCV(solver='liblinear', cv=10, random_state=908)
lr.fit(X_train, np.ravel(y_train))

LogisticRegressionCV(cv=10, random_state=908, solver='liblinear')

In [ ]:
#@title Métricas modelo base

y_pred_proba = lr.predict_proba(X_test)[:,1]
y_pred = y_pred_proba >= 0.5

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"F1: {f1_score(y_test, y_pred)}")

Accuracy: 0.735
Recall: 0.8613138686131386
F1: 0.8166089965397924


### Sesgo del modelo




In [ ]:
# Para facilitar el proceso

def get_aif_metrics(attr, df):

  privileged_groups = [{attr: 1}]
  unprivileged_groups = [{attr: 0}]

  attributes_params = dict(protected_attribute_names=[attr], label_names=['credit-score'])
  dt = BinaryLabelDataset(df=df, **attributes_params)

  metric = BinaryLabelDatasetMetric(dt,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)

  return metric, dt

#### Discriminación por sexo

In [ ]:
metric_age_train, train_aifdf = get_aif_metrics('age_cat', df_train)
print(f'Train Statistical Parity Difference (age): {metric_age_train.mean_difference()}')
print(f'Train Disparate Impact (age): {metric_age_train.disparate_impact()}')


Train Statistical Parity Difference (age): -0.13278706925580863
Train Disparate Impact (age): 0.8178037886955184


In [ ]:
y_train_pred_proba = lr.predict_proba(X_train)[:,1]
y_train_pred = y_train_pred_proba >= 0.5

In [ ]:
df_pred = X_train.copy()
df_pred['credit-score'] = y_train_pred

In [ ]:
metric_age_pred, pred_aifdf = get_aif_metrics('age_cat', df_pred)
print(f'Pred Statistical Parity Difference (age): {metric_age_pred.mean_difference()}')
print(f'Pred Disparate Impact (age): {metric_age_pred.disparate_impact()}')

Pred Statistical Parity Difference (age): -0.18487943754528113
Pred Disparate Impact (age): 0.7710176431929628


In [ ]:
orig_vs_pred = ClassificationMetric(train_aifdf, pred_aifdf,
                                                    unprivileged_groups=unprivileged_groups,
                                                    privileged_groups=privileged_groups)

In [ ]:
print(f'Error rate difference (unprivileged error rate - privileged error rate)= {orig_vs_pred.error_rate_difference()}\n')

print(f'False negative rate for privileged groups = {orig_vs_pred.false_negative_rate(privileged=True)}')
print(f'False negative rate for unprivileged groups = {orig_vs_pred.false_negative_rate(privileged=False)}')
print(f'False negative rate ratio = {orig_vs_pred.false_negative_rate_ratio()}\n')

print(f'False positive rate for privileged groups = {orig_vs_pred.false_positive_rate(privileged=True)}')
print(f'False positive rate for unprivileged groups = {orig_vs_pred.false_positive_rate(privileged=False)}')
print(f'False positive rate ratio = {orig_vs_pred.false_positive_rate_ratio()}')

Error rate difference (unprivileged error rate - privileged error rate)= 0.10803171460933281

False negative rate for privileged groups = 0.07188160676532769
False negative rate for unprivileged groups = 0.2222222222222222
False negative rate ratio = 3.0915032679738563

False positive rate for privileged groups = 0.48295454545454547
False positive rate for unprivileged groups = 0.39344262295081966
False positive rate ratio = 0.8146576663452266
